In [ ]:
! pip install farm-haystack
!pip install PyPDF2
!pip install wget

In [ ]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [ ]:
import os
doc_dir = "data/article"
if not os.path.exists(doc_dir):
        os.makedirs(doc_dir)

In [ ]:
import wget
wget.download(url='https://docsmsftpdfs.blob.core.windows.net/guides/azure/azure-ops-guide.pdf', out=doc_dir)

'data/article/azure-ops-guide.pdf'

In [ ]:
import PyPDF2
article=[]
pdfFileObj = open('data/article/azure-ops-guide.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
for page in range(pdfReader.numPages):
  pageObj = pdfReader.getPage(page)
  article.append(pageObj.extractText())
article_text = open('data/article/azure-ops-guide.txt',"w")
article_text.writelines(article)

In [ ]:
write_documents_to_db(document_store=document_store, document_dir=doc_dir, clean_func=clean_wiki_text, only_empty_db=True)

06/22/2020 15:07:46 - INFO - haystack.indexing.io -   Wrote 1 docs to DB


In [ ]:
from haystack.retriever.tfidf import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)
finder = Finder(reader, retriever)

In [ ]:
prediction = finder.get_answers(question="What is Azure Portal?", top_k_retriever=10, top_k_reader=5)

06/22/2020 15:08:32 - INFO - haystack.retriever.tfidf -   Identified 1 candidates via retriever:
  paragraph_id                       document_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

OrderedDict([(0, array([0.26915871]))])


Inferencing Samples: 100%|██████████| 2/2 [00:57<00:00, 28.80s/ Batches]


In [ ]:
print_answers(prediction, details="minimal")

[   {   'answer': 'a web-based application that can be used to create, manage, '
                  'and remove Azure\n'
                  'resources and services',
        'context': 'ries.\n'
                   'The Azure portal is a web-based application that can be '
                   'used to create, manage, and remove Azure\n'
                   'resources and services. The Azure portal is loc'},
    {   'answer': 'requires only an active Azure subscription\n'
                  'and access to a web browser',
        'context': 'rtual network by using the Azure portal requires only an '
                   'active Azure subscription\n'
                   'and access to a web browser. You can deploy a new virtual '
                   'network i'},
    {   'answer': 'provides detailed insight into Azure billing information',
        'context': ' to view billing information. The Azure portal\n'
                   'provides detailed insight into Azure billing information.\n'
          